In [1]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [2]:
# Open the netCDF file
nc_file = nc.Dataset('fordle/ela_data/train/ALP00_A78_C0/ex.nc', 'r')
# Get the number of records in the file
num_records = nc_file.dimensions['idx'].size

In [3]:
# Define the index to split the file
split_index = int(num_records*0.8)  # Change this to your desired split index

In [4]:
# Create two separate netCDF files
part1_file = nc.Dataset('part1.nc', 'w', format='NETCDF4')
part2_file = nc.Dataset('part2.nc', 'w', format='NETCDF4')


In [5]:
# Copy global attributes from the original file
part1_file.setncatts(nc_file.__dict__)
part2_file.setncatts(nc_file.__dict__)

In [6]:
# Create dimensions in the new files 
exclude=['x','y','idx']
for name, dimension in nc_file.dimensions.items():
    if name == 'idx':
        part1_file.createDimension(name, split_index)
        part2_file.createDimension(name, num_records - split_index)
    else:
        part1_file.createDimension(name, len(dimension))
        part2_file.createDimension(name, len(dimension))


In [7]:
# Copy variables from the original file to the new files
for name, variable in nc_file.variables.items():
    if name not in exclude:
        # Create variables in the new files
        var1 = part1_file.createVariable(name, variable.dtype, variable.dimensions)
        var2 = part2_file.createVariable(name, variable.dtype, variable.dimensions)
        
        
        # Copy variable data based on the split index
        part1_file[name][:] = nc_file[name][:split_index] 
        part2_file[name][:] = nc_file[name][split_index:] 

  

In [ ]:
# Copy variables from the original file to the new files
for name, variable in nc_file.variables.items():
    if name in ['ela', 'topg', 'footprint']:
        # Create variables in the new files
        var1 = part1_file.createVariable(name, variable.dtype, variable.dimensions)
        var2 = part2_file.createVariable(name, variable.dtype, variable.dimensions)

        # Copy variable attributes
        var1.setncatts(variable.__dict__)
        var2.setncatts(variable.__dict__)

        # Copy variable data based on the split index
        var1[:] = variable[:split_index]
        var2[:] = variable[split_index:]


In [8]:
# Close the files
nc_file.close()
part1_file.close()
part2_file.close()